In [ ]:
import numpy as np 

In [ ]:
from energyclustering.sampling.preprocessing import DataPreprocessor
from dask.distributed import Client
import pandas as pd
import altair as alt

In [ ]:
%load_ext autoreload
%autoreload 2

# The data

In [ ]:
daily_data_df, data_df, daily_info_df, weather_df = (
    DataPreprocessor()
    .preprocess_info_df('baseline')
    .preprocess_weather_df('baseline')
    .drop_days_with_nan(True)
    .subsample_days(week_reduction_factor = 2)
    # for testing only!
    .subsample_years(300)
    .get_data()
)
daily_data_df.shape

# Folds

In [ ]:
generator = np.random.default_rng(1)
shuffled = data_df.index.to_numpy(copy=True)
generator.shuffle(shuffled)
folds = np.array_split(shuffled, 3)

# The samplers

In [ ]:
from energyclustering.sampling.samplers import MetadataSampler
from energyclustering.sampling.samplers import RandomSamplerBaseline, RandomSampler
from energyclustering.sampling.day_of_year_samplers import RandomDayFromYearSampler, SpecificDayFromYearSampler, SimilarDayFromYearSampler
from sklearn.cluster import KMeans

In [ ]:
NB_CLUSTER = 20 
NB_SIMILAR_WEATHER_DAYS = 10

# Evaluate the samplers

In [ ]:
from energyclustering.sampling.evaluation import SamplerEvaluator
from pathlib import Path
import time

In [ ]:
NB_CLUSTERS = 20
NB_SIMILAR_WEATHER_DAYS = 10
models = dict(
    random_day_random_year = RandomSamplerBaseline(250), 
    random_day_from_metadata_year = RandomDayFromYearSampler(MetadataSampler(KMeans(NB_CLUSTERS)), 20), 
    weather_day_from_random_year = SimilarDayFromYearSampler(RandomSampler(), NB_SIMILAR_WEATHER_DAYS, weather_df), 
    weather_day_from_metadata_year = SimilarDayFromYearSampler(MetadataSampler(KMeans(NB_CLUSTERS)), NB_SIMILAR_WEATHER_DAYS, weather_df)
)
result_path = Path()/'results'/'energyville_ablation'
with Client(local_directory = '/cw/dtailocal/', n_workers=20, threads_per_worker = 1) as client: 
    evaluator = SamplerEvaluator(folds, daily_data_df, daily_info_df, data_df, client, 200)
    for key, model in models.items(): 
        start_time = time.time()
        evaluator.evaluate_and_save(model, result_path/f"{key}_v0.pkl")
        print(f"{key}: {time.time()- start_time} s")
        client.restart()


In [ ]:

all_files = list(result_path.iterdir())
results = pd.concat([pd.read_pickle(path) for path in all_files], keys = [path.stem for path in all_files], axis = 1)
results

In [ ]:
results.mean(axis = 0)

In [ ]:
results.std(axis = 0)